In [2]:
import pandas as pd
import numpy as np

In [3]:
#change the dataset here
path = 'MaterSlave.xlsx'

In [4]:
#open the sheets in the excel file
df = pd.read_excel(open(path, 'rb'), sheet_name='Master+Slave')
df_Master_seeds = pd.read_excel(open(path, 'rb'), sheet_name='Master')

In [5]:
#identify the size of dataset
df.shape

(42454, 4)

In [6]:
#identify the columns
df.columns

Index(['ID User', 'ID Master', 'Seats', 'Status'], dtype='object')

In [7]:
#identify the user status
df = df[(df.Status == 'Master') | (df.Status == 'Slave')]

In [8]:

df_Master = df[df.Status == 'Master'].copy()[['ID User','Seats']]
df_Slave = df[df.Status == 'Slave'].copy()

In [9]:
#convert the datatype of "ID User"
df_Slave['ID User'] = df_Slave['ID User'].astype('str')

In [10]:

df_Master = df_Master.rename(columns={'ID User': 'ID Master'})

In [11]:

df_t = df_Slave.groupby('ID Master', as_index=False).agg({'Seats': 'sum', 'ID User': lambda x: ' '.join(x)})

In [12]:
df_Master.head()

,ID Master,Seats
7,2108523,50
8,1211251,1000
22,2025730,50
37,2104688,50
38,2105576,50


In [13]:
#count how many slaves in one master account
df_t['Slave_count'] = df_t['ID User'].apply(lambda x: x.count(' ')+1)

In [14]:
#change the datatype of "ID Master"
df_t['ID Master'] = df_t['ID Master'].astype('int64')

In [15]:
#see the new dataset
df_t.head()

,ID Master,Seats,ID User,Slave_count
0,88813,1034,10046 23094 26292 30237 35291 38085 40040 4153...,1216
1,138730,2,669809 669839 669836 669833 669831 669828 6698...,20
2,142922,1,1285693 1347421 1632672 1902688 2072695 1417091,6
3,149642,50,2081909 1288620,2
4,189053,75,442781 744765 748312 829592 862432 877001 9094...,107


In [16]:
#see the master seats
df_total_seats = df_t.merge(df_Master, on = ['ID Master'])

In [17]:
#calculate the totle seats
df_total_seats['Seats'] = df_total_seats.Seats_x + df_total_seats.Seats_y

In [18]:
df_total_seats.head()

,ID Master,Seats_x,ID User,Slave_count,Seats_y,Seats
0,138730,2,669809 669839 669836 669833 669831 669828 6698...,20,20,22
1,142922,1,1285693 1347421 1632672 1902688 2072695 1417091,6,9,10
2,149642,50,2081909 1288620,2,3,53
3,194312,0,686472 1300699 1472320 1472321 1575122 1614128...,13,14,14
4,196908,0,1703417,1,4,4


In [19]:
#drop unnecessary columns for seats of different status and only leave the necessary one be in the dataset
df_total_seats.drop(['Seats_x', 'Seats_y'], axis=1,inplace=True)

In [20]:
df_total_seats.head()

,ID Master,ID User,Slave_count,Seats
0,138730,669809 669839 669836 669833 669831 669828 6698...,20,22
1,142922,1285693 1347421 1632672 1902688 2072695 1417091,6,10
2,149642,2081909 1288620,2,53
3,194312,686472 1300699 1472320 1472321 1575122 1614128...,13,14
4,196908,1703417,1,4


In [21]:
df_total_seats.dtypes

ID Master       int64
ID User        object
Slave_count     int64
Seats           int64
dtype: object

In [22]:
#Rename the column name
df_Master_seeds = df_Master_seeds.rename(columns={'Master ID': 'ID Master'})

In [23]:
df_Master_seeds.dtypes

ID Master    int64
dtype: object

In [24]:
#merge the sheet one and sheet two
df = df_Master_seeds.merge(df_total_seats, on=['ID Master'], how='outer')

In [25]:
#export the dataset
df.to_csv('Master_Slave_result.csv', index=True)